In [1]:


import cv2
from typing import List, Generator, Optional

def frame_generator(video_paths: List[str]) -> Generator[List[Optional[cv2.Mat]], None, None]:
    """
    从多个视频文件生成帧。

    参数:
        video_paths (List[str]): 视频文件路径列表。

    生成:
        List[Optional[cv2.Mat]]: 每个视频中的帧列表，如果视频结束则为 None。
    """
    # 打开每个视频文件
    caps = [cv2.VideoCapture(path) for path in video_paths]

    while True:
        frames = []
        all_frames_read = True
        for cap in caps:
            ret, frame = cap.read()
            if not ret:
                frames.append(None)
                all_frames_read = False
            else:
                frames.append(frame)
                
        # 如果所有视频都读取结束，则退出循环
        if not all_frames_read:
            break

        yield frames

    # 释放所有视频文件
    for cap in caps:
        cap.release()

# 使用示例
video_paths = ["../data/testolabc1.avi", "../data/testolabc2.avi", ]#"../data/testolabc3.avi", "../data/testolabc4.avi"]
gen = frame_generator(video_paths)

for frames in gen:
    # 处理提取的帧，如拼接视频等
    if all(frame is not None for frame in frames):
        # 将多帧水平拼接
        concatenated_frame = cv2.hconcat(frames)
        # 在这里处理拼接后的帧，如保存到视频文件
        # 例如，使用 cv2.VideoWriter 写入新视频文件
        cv2.imwrite("concatenated_frame.jpg", concatenated_frame)


In [2]:
import numpy as np
def calculate_homography(frames: List[cv2.Mat]) -> Optional[np.ndarray]:
    """
    计算图像之间的单应性矩阵。

    参数:
        frames (List[cv2.Mat]): 图像列表。

    返回:
        Optional[np.ndarray]: 单应性矩阵，如果计算失败则为 None。
    """
    if len(frames) < 2:
        raise ValueError("需要至少两幅图像来计算单应性矩阵")

    # 使用ORB检测关键点和描述符
    orb = cv2.ORB_create()

    # 提取第一张图像的关键点和描述符
    kp1, des1 = orb.detectAndCompute(frames[0], None)

    homographies = []

    for i in range(1, len(frames)):
        # 提取下一张图像的关键点和描述符
        kp2, des2 = orb.detectAndCompute(frames[i], None)

        # 使用BFMatcher进行特征匹配
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des1, des2)

        # 排序匹配结果
        matches = sorted(matches, key=lambda x: x.distance)

        # 获取匹配点的坐标
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)

        # 计算单应性矩阵
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if H is not None:
            homographies.append(H)

        # 更新关键点和描述符
        kp1, des1 = kp2, des2

    if not homographies:
        return None
    
    # 返回计算的单应性矩阵
    return homographies

In [3]:

video_paths = ["../data/testolabc1.avi", "../data/testolabc2.avi"]
gen = frame_generator(video_paths)

count = 0
frame_count = 180
homography_matrices = []
current_H = None

m_count  = 10
strength_m = 0.7

for frames in gen:
    if count < frame_count:
        count += 1
        # 过滤掉 None 帧
        valid_frames = [frame for frame in frames if frame is not None]
        if len(valid_frames) >= 2 and count % m_count == 0:
            new_homographies = calculate_homography(valid_frames)
            if new_homographies:
                new_H = new_homographies[0]  # 假设只处理第一对图像的单应性矩阵
                if current_H is None:
                    current_H = new_H
                else:
                    current_H = (1-strength_m) * new_H + (strength_m) * current_H
            else:
                print("无法计算单应性矩阵")
        
        # 如果 current_H 仍为 None，则跳过此步骤
        if current_H is not None:
            homography_matrices.append(current_H)
        else:
            homography_matrices.append(np.eye(3))  # 使用单位矩阵作为初始值
    else:
        break

# 如果最终的长度不足180，则补齐到180
while len(homography_matrices) < frame_count:
    homography_matrices.append(current_H if current_H is not None else np.eye(3))

print("最终的单应性矩阵列表：", len(homography_matrices), homography_matrices)

g++ -o video_stitching video_stitching.cpp `pkg-config --cflags --libs opencv4`
./video_stitch ../data/testolabc1.avi ../data/testolabc2.avi ./output

最终的单应性矩阵列表： 180 [array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([[ 4.46808864e-01, -4.80906481e-02, -1.55081992e+02],
       [-1.71459809e-01,  4.85951029e-01,  8.32799624e+01],
       [-9.97772325e-04, -3.73400224e-05,  1.00000000e+00]]), array([[ 4.46808864e-01, -4.80906481e-02, -1.55081992e+02],
       [-1.71459809e-01,  4.85951029e-01,  8.32799624e+01],
       [-9.97772325e-04, -3.73400224e-05,  1.00000000e+00]]), array([[ 4.46808864e-01, -4.809064

In [16]:





def stitch_images(img1, img2, homography_matrix, output_path="result.jpg"):
    # 获取图像的尺寸
    h1, w1, _ = img1.shape
    h2, w2, _ = img2.shape

    # 计算变换后的图像范围
    points_img2 = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
    points_img2_transformed = cv2.perspectiveTransform(points_img2, homography_matrix)
    points = np.concatenate((points_img2_transformed, np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2)), axis=0)

    # 计算变换后的图像的尺寸
    [x_min, y_min] = np.int32(points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(points.max(axis=0).ravel() + 0.5)
    translation_dist = [-x_min, -y_min]
    
    h_stitched, w_stitched = y_max - y_min, x_max - x_min

    # 将第二张图像变换到拼接后的图像平面
    homography_matrix_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]]) @ homography_matrix
    img2_warped = cv2.warpPerspective(img2, homography_matrix_translation, (w_stitched, h_stitched))

    # 创建拼接图像的空白区域
    stitched_image = img2_warped.copy()
    stitched_image[translation_dist[1]:h1+translation_dist[1], translation_dist[0]:w1+translation_dist[0]] = img1

    # 保存拼接后的图像
    cv2.imwrite(output_path, stitched_image)
    print(f"Stitched image saved as {output_path}")




In [17]:

gen = frame_generator(video_paths)
img1, img2 = next(gen)
test1 = stitch_images(img1, img2, homography_matrices[40])


Stitched image saved as result.jpg


In [6]:
cv2.imwrite("test1.jpg", test1)

error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
